<a href="https://colab.research.google.com/github/Boyakin/qChem/blob/main/%D0%9A%D0%B2%D0%A5%D0%B8%D0%BCPractice2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyscf geometric pyberny

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
import numpy as np
from pyscf import gto, scf, ci, mcscf, mp, mrpt, cc, dft, eph
import time

In [ ]:
rco = 1.21
rch = 1.11
ahch = 120.

geom_zmat = f'''
O
C   1   {rco}
H   2   {rch}   1   {ahch}
H   2   {rch}   1   {ahch}  3 180.
'''

In [ ]:
hcoh = gto.Mole(atom=geom_zmat, unit='Angstrom', basis='ccpvtz', charge=0, spin=0, symmetry=True)
hcoh.build()

In [ ]:
start_hf = time.time()
hf_hcoh = scf.RHF(hcoh).run()
print(f'HF done in {(time.time()-start_hf):.3f} seconds')
start_mp2 = time.time()
mp2_hcoh = mp.MP2(hf_hcoh).run()
print(f'MP2 done in {(time.time()-start_mp2):.3f} seconds')

converged SCF energy = -113.910581741567
HF done in 1.216 seconds
E(MP2) = -114.33453010862  E_corr = -0.423948367053766
E(SCS-MP2) = -114.332667436716  E_corr = -0.422085695149388
MP2 done in 0.271 seconds


In [ ]:
start_casscf = time.time()
casscf_hcoh = mcscf.CASSCF(hf_hcoh, 8, 8).run()
print(f'CASSCF done in {(time.time()-start_casscf):.3f} seconds')
start_casci = time.time()
casci_hcoh = mcscf.CASCI(hf_hcoh, 8, 8).run()
print(f'CASCI done in {(time.time()-start_casci):.3f} seconds')
start_nevpt = time.time()
nevpt_hcoh = mrpt.NEVPT(casci_hcoh).run()
print(f'NEVPT done in {(time.time()-start_nevpt):.3f} seconds')

CASSCF energy = -114.029034520586
CASCI E = -114.029034520586  E(CI) = -19.3188123518140  S^2 = 0.0000000
CASSCF done in 26.776 seconds
CASCI E = -113.931239777809  E(CI) = -17.8796770166617  S^2 = 0.0000000
CASCI done in 0.421 seconds
Sr    (-1)',   E = -0.04733449238647
Si    (+1)',   E = -0.00410823157894
Sijrs (0)  ,   E = -0.04062696139142
Sijr  (+1) ,   E = -0.00579492569052
Srsi  (-1) ,   E = -0.10595089396308
Srs   (-2) ,   E = -0.14539755396260
Sij   (+2) ,   E = -0.00111512040541
Sir   (0)' ,   E = -0.03785847029974
Nevpt2 Energy = -0.388186649678188
NEVPT done in 3.085 seconds


In [ ]:
nevpt_hcoh.e_corr + casci_hcoh.e_tot

np.float64(-114.31942642748717)

In [ ]:
start_ccsd = time.time()
ccsd_hcoh = cc.CCSD(hf_hcoh).run()
ccsd_time = time.time() - start_ccsd
print(f'CCSD done in {(ccsd_time):.3f} seconds')
ccsdt_hcoh = ccsd_hcoh.ccsd_t()

E(CCSD) = -114.3448634734393  E_corr = -0.4342817318725921
CCSD done in 9.525 seconds
CCSD(T) correction = -0.0170234449885299


In [ ]:
start_b3lyp = time.time()
b3lyp_hcoh = dft.RKS(hcoh, xc='B3LYP').run()
print(f'B3LYP done in {(time.time() - start_b3lyp):.3f} seconds')

start_pbe = time.time()
pbe_hcoh = dft.RKS(hcoh, xc='PBE').run()
print(f'PBE done in {(time.time() - start_pbe):.3f} seconds')

start_r2scan = time.time()
r2scan_hcoh = dft.RKS(hcoh, xc='r2SCAN').run()
print(f'r2SCAN done in {(time.time() - start_r2scan):.3f} seconds')

converged SCF energy = -114.548752174448
B3LYP done in 4.254 seconds
converged SCF energy = -114.413625663887
PBE done in 4.873 seconds
converged SCF energy = -114.480920535033
r2SCAN done in 5.065 seconds


In [ ]:
# hcoh_hf_opt = hf_hcoh.Gradients().optimizer(solver='geomeTRIC').kernel()
start_hf = time.time()
hf_hcoh.Gradients().run()
print(f'HF gradient done in {(time.time()-start_hf):.3f} seconds')

start_mp2 = time.time()
mp2_hcoh.Gradients().run()
print(f'MP2 gradient done in {(time.time()-start_mp2):.3f} seconds')

start_casci = time.time()
casci_hcoh.Gradients().run()
print(f'CASCI gradient done in {(time.time()-start_casci):.3f} seconds')

start_ccsd = time.time()
ccsd_hcoh.Gradients().run()
print(f'CCSD gradient done in {(time.time() - start_ccsd):.3f} seconds')

start_b3lyp = time.time()
b3lyp_hcoh.Gradients().run()
print(f'B3LYP gradient done in {(time.time() - start_b3lyp):.3f} seconds')

start_pbe = time.time()
pbe_hcoh.Gradients().run()
print(f'PBE gradient done in {(time.time() - start_pbe):.3f} seconds')

start_r2scan = time.time()
r2scan_hcoh.Gradients().run()
print(f'r2SCAN gradient done in {(time.time() - start_r2scan):.3f} seconds')

--------------- SymAdaptedRHF gradients ---------------
         x                y                z
0 O    -0.0562005608     0.0000000000    -0.0000000000
1 C     0.0519331546     0.0000000000     0.0000000000
2 H     0.0021337031     0.0000000000     0.0148656312
3 H     0.0021337031    -0.0000000000    -0.0148656312
----------------------------------------------
HF gradient done in 3.759 seconds
--------------- MP2 gradients ---------------
         x                y                z
0 O    -0.0017587539     0.0000000000    -0.0000000000
1 C     0.0034695506     0.0000000000     0.0000000000
2 H    -0.0008553984     0.0000000000     0.0112069176
3 H    -0.0008553984    -0.0000000000    -0.0112069176
----------------------------------------------
MP2 gradient done in 10.668 seconds
--------- SymAdaptedCASCI gradients for state 0 ----------
         x                y                z
0 O    -0.0231835694     0.0000000000    -0.0000000000
1 C     0.0214679449     0.0000000000     0.0

In [ ]:
hcoh_hf_opt_geomtric = hf_hcoh.Gradients().optimizer(solver='geomeTRIC').kernel()

geometric-optimize called with the following command line:
/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-5a1718ce-10f0-4cf6-bc84-581707384c7f.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   C   1.210000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   1.765000   0.000000   0.961288    0.000000  0.000000  0.000000
   H   1.765000  -0.000000  -0.961288    0.000000  0.000000  0.000000
converged SCF energy = -113.910581741567
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0562006744     0.0000000000    -0.0000000000
1 C     0.0519332623     0.0000000000     0.0000000000
2 H     0.0021337061     0.0000000000     0.0148656333
3 H     0.0021337061    -0.0000000000    -0.0148656333
----------------------------------------------
cycle 1: E = -113.910581742  dE = -113.911  norm(grad) = 0.0794145


Step    0 : Gradient = 3.971e-02/5.620e-02 (rms/max) Energy = -113.9105817416
Hessian Eigenvalues: 4.50000e-02 5.00000e-02 5.00000e-02 ... 3.26000e-01 3.26000e-01 9.99547e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.011187   0.000000  -0.016749    0.011187  0.000000 -0.016749
   C   1.191113  -0.000000   0.010719   -0.018887 -0.000000  0.010719
   H   1.781132   0.000000   0.924093    0.016132  0.000000 -0.037195
   H   1.756568   0.000000  -0.918063   -0.008432  0.000000  0.043225
converged SCF energy = -113.91317874453
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0032624697     0.0000000000    -0.0000000000
1 C     0.0063346838     0.0000000000     0.0000000000
2 H    -0.0015361070    -0.0000000000    -0.0034011262
3 H    -0.0015361070     0.0000000000     0.0034011262
----------------------------------------------
cycle 2: E = -113.913178745  dE = -0.002597  norm(grad) = 0.00886716


Step    1 : Displace = 3.339e-02/4.404e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 4.434e-03/6.335e-03 (rms/max) E (change) = -113.9131787445 (-2.597e-03) Quality = 0.997
Hessian Eigenvalues: 4.50000e-02 4.99990e-02 5.00000e-02 ... 3.26000e-01 3.65512e-01 9.12350e-01



Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.012515   0.000000  -0.018129    0.001328  0.000000 -0.018129
   C   1.190268  -0.000000   0.010064   -0.000845 -0.000000  0.010064
   H   1.778260   0.000000   0.931142    0.009410  0.000000  0.010064
   H   1.758957   0.000000  -0.923077   -0.009893  0.000000 -0.001999
converged SCF energy = -113.913224484382
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O     0.0005920410     0.0000000000     0.0000000000
1 C     0.0000154018     0.0000000000     0.0000000000
2 H    -0.0003037214     0.0000000000     0.0000374281
3 H    -0.0003037214    -0.0000000000    -0.0000374281
----------------------------------------------
cycle 3: E = -113.913224484  dE = -4.57399e-05  norm(grad) = 0.000733515


Step    2 : Displace = 4.397e-03/6.223e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 3.668e-04/5.920e-04 (rms/max) E (change) = -113.9132244844 (-4.574e-05) Quality = 0.968
Hessian Eigenvalues: 4.50000e-02 5.00000e-02 5.00000e-02 ... 3.25998e-01 3.48609e-01 9.52211e-01



Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.011486   0.000000  -0.018702   -0.001029  0.000000 -0.018702
   C   1.189484  -0.000000   0.010512   -0.000784 -0.000000  0.010512
   H   1.779757   0.000000   0.930540    0.011148  0.000000  0.003430
   H   1.759273   0.000000  -0.922350   -0.009335  0.000000  0.004759
converged SCF energy = -113.91322532555
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0001612829     0.0000000000    -0.0000000000
1 C    -0.0000255971     0.0000000000    -0.0000000000
2 H     0.0000934400     0.0000000000     0.0000178359
3 H     0.0000934400    -0.0000000000    -0.0000178359
----------------------------------------------
cycle 4: E = -113.913225326  dE = -8.41168e-07  norm(grad) = 0.000211579


Step    3 : Displace = 1.161e-03/1.505e-03 (rms/max) Trust = 2.000e-01 (+) Grad = 1.058e-04/1.613e-04 (rms/max) E (change) = -113.9132253255 (-8.412e-07) Quality = 0.729
Hessian Eigenvalues: 4.50000e-02 5.00000e-02 5.00000e-02 ... 3.25998e-01 3.48609e-01 9.52211e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 20.095 seconds


In [ ]:
print(hcoh_hf_opt_geomtric.tostring())

O           0.01149        0.00000        0.00000
C           1.18948        0.00000        0.00000
H           1.76952        0.00000        0.92645
H           1.76952       -0.00000       -0.92645


In [ ]:
hcoh_hf_opt_berny = hf_hcoh.Gradients().optimizer(solver='berny').kernel()

0 Internal coordinates:0 * Number of fragments: 10 * Number of internal coordinates: 60 * Number of strong bonds: 30 * Number of strong angles: 3


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   C   1.210000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   1.765000   0.000000   0.961288    0.000000  0.000000  0.000000
   H   1.765000  -0.000000  -0.961288    0.000000  0.000000  0.000000
converged SCF energy = -113.910581741567
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0562006744     0.0000000000    -0.0000000000
1 C     0.0519332623     0.0000000000     0.0000000000
2 H     0.0021337061     0.0000000000     0.0148656333
3 H     0.0021337061    -0.0000000000    -0.0148656333
----------------------------------------------
cycle 1: E = -113.910581742  dE = -113.911  norm(grad) = 0.0794145


1 Energy: -113.9105817421 Pure RFO step was performed:1 * Trust radius: 0.31 * Number of negative eigenvalues: 01 * Lowest eigenvalue: 0.171 * lambda: -0.007051 Quadratic step: RMS: 0.05, max: 0.1021 * Predicted energy change: -0.003581 Total step: RMS: 0.05, max: 0.1021 Perfect transformation to cartesians in 4 iterations1 * RMS(dcart): 5.86e-08, RMS(dq): 1.77e-071 Convergence criteria:1 * Gradient RMS: 0.0246 > 0.00015 => no1 * Gradient maximum: 0.0562 > 0.00045 => no1 * Step RMS: 0.05 > 0.0012 => no1 * Step maximum: 0.102 > 0.0018 => no


Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.033848  -0.000000  -0.000000    0.033848 -0.000000 -0.000000
   C   1.190071   0.000000   0.000000   -0.019929  0.000000  0.000000
   H   1.758040   0.000000   0.935483   -0.006960 -0.000000 -0.025805
   H   1.758040  -0.000000  -0.935483   -0.006960  0.000000  0.025805
converged SCF energy = -113.912140349367
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O     0.0484214073     0.0000000000     0.0000000000
1 C    -0.0409032889     0.0000000000    -0.0000000000
2 H    -0.0037590592     0.0000000000     0.0018227956
3 H    -0.0037590592    -0.0000000000    -0.0018227956
----------------------------------------------
cycle 2: E = -113.912140349  dE = -0.00155861  norm(grad) = 0.0636602


2 Energy: -113.9121403492 Hessian update information:2 * Change: RMS: 0.0845, max: 0.4992 Trust update: Fletcher's parameter: 0.4362 Linear interpolation:2 * Energies: -113.91214, -113.910582 * Derivatives: -0.00456, 0.007052 * Quartic interpolation was performed: t = 0.3552 * Interpolated energy: -113.912942 Pure RFO step was performed:2 * Trust radius: 0.32 * Number of negative eigenvalues: 02 * Lowest eigenvalue: 0.1542 * lambda: -0.0006092 Quadratic step: RMS: 0.0217, max: 0.04032 * Predicted energy change: -0.0003052 Total step: RMS: 0.0221, max: 0.04492 Perfect transformation to cartesians in 3 iterations2 * RMS(dcart): 3.53e-07, RMS(dq): 9.45e-072 Convergence criteria:2 * Gradient RMS: 0.02 > 0.00015 => no2 * Gradient maximum: 0.0484 > 0.00045 => no2 * Step RMS: 0.0221 > 0.0012 => no2 * Step maximum: 0.0449 > 0.0018 => no


Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.010553  -0.000000  -0.000000   -0.023295 -0.000000 -0.000000
   C   1.190560   0.000000   0.000000    0.000488  0.000000  0.000000
   H   1.769444   0.000000   0.927478    0.011403 -0.000000 -0.008005
   H   1.769444  -0.000000  -0.927478    0.011403  0.000000  0.008005
converged SCF energy = -113.91321698168
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0040358370     0.0000000000    -0.0000000000
1 C     0.0037664485     0.0000000000     0.0000000000
2 H     0.0001346942     0.0000000000     0.0004232147
3 H     0.0001346942    -0.0000000000    -0.0004232147
----------------------------------------------
cycle 3: E = -113.913216982  dE = -0.00107663  norm(grad) = 0.00555595


3 Energy: -113.9132169823 Hessian update information:3 * Change: RMS: 0.0159, max: 0.0923 Trust update: Fletcher's parameter: 3.533 Linear interpolation:3 * Energies: -113.91322, -113.912143 * Derivatives: -0.000174, 0.002393 * Quartic interpolation was performed: t = 0.07263 * Interpolated energy: -113.913223 Pure RFO step was performed:3 * Trust radius: 0.33 * Number of negative eigenvalues: 03 * Lowest eigenvalue: 0.1523 * lambda: -3.99e-063 Quadratic step: RMS: 0.00199, max: 0.003823 * Predicted energy change: -1.99e-063 Total step: RMS: 0.00185, max: 0.003613 Perfect transformation to cartesians in 2 iterations3 * RMS(dcart): 2.18e-07, RMS(dq): 7.33e-073 Convergence criteria:3 * Gradient RMS: 0.00167 > 0.00015 => no3 * Gradient maximum: 0.00404 > 0.00045 => no3 * Step RMS: 0.00185 > 0.0012 => no3 * Step maximum: 0.00361 > 0.0018 => no


Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.011634   0.000000   0.000000    0.001081  0.000000  0.000000
   C   1.189728   0.000000   0.000000   -0.000832  0.000000  0.000000
   H   1.769319   0.000000   0.926521   -0.000124 -0.000000 -0.000956
   H   1.769319  -0.000000  -0.926521   -0.000124  0.000000  0.000956
converged SCF energy = -113.91322535222
--------------- SymAdaptedRHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0002784424     0.0000000000    -0.0000000000
1 C     0.0003083093     0.0000000000     0.0000000000
2 H    -0.0000149334    -0.0000000000    -0.0000252106
3 H    -0.0000149334     0.0000000000     0.0000252106
----------------------------------------------
cycle 4: E = -113.913225352  dE = -8.37054e-06  norm(grad) = 0.000417495


4 Energy: -113.9132253524 Hessian update information:4 * Change: RMS: 0.0138, max: 0.07994 Trust update: Fletcher's parameter: 4.24 Linear interpolation:4 * Energies: -113.91323, -113.913224 * Derivatives: 9.55e-07, 1.58e-054 * Quartic interpolation was performed: t = -0.06414 * Interpolated energy: -113.913234 Pure RFO step was performed:4 * Trust radius: 0.34 * Number of negative eigenvalues: 04 * Lowest eigenvalue: 0.1534 * lambda: -1.88e-084 Quadratic step: RMS: 8.98e-05, max: 0.0001364 * Predicted energy change: -9.4e-094 Total step: RMS: 0.000123, max: 0.0002724 Perfect transformation to cartesians in 2 iterations4 * RMS(dcart): 4.14e-10, RMS(dq): 1.07e-094 Convergence criteria:4 * Gradient RMS: 0.000115 < 0.00015 => OK4 * Gradient maximum: 0.000278 < 0.00045 => OK4 * Step RMS: 0.000123 < 0.0012 => OK4 * Step maximum: 0.000272 < 0.0018 => OK4 * All criteria matched

In [ ]:
hcoh_b3lyp_opt = b3lyp_hcoh.Gradients().optimizer(solver='geomeTRIC').kernel()

geometric-optimize called with the following command line:
/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-5a1718ce-10f0-4cf6-bc84-581707384c7f.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   C   1.210000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   1.765000   0.000000   0.961288    0.000000  0.000000  0.000000
   H   1.765000  -0.000000  -0.961288    0.000000  0.000000  0.000000
converged SCF energy = -114.54875217442
--------------- SymAdaptedRKS_Scanner gradients ---------------
         x                y                z
0 O    -0.0138520193     0.0000000000    -0.0000000000
1 C     0.0210254167     0.0000000000     0.0000000000
2 H    -0.0035823315     0.0000000000     0.0061338876
3 H    -0.0035823315    -0.0000000000    -0.0061338876
----------------------------------------------
cycle 1: E = -114.548752174  dE = -114.549  norm(grad) = 0.0271083


Step    0 : Gradient = 1.355e-02/2.103e-02 (rms/max) Energy = -114.5487521744
Hessian Eigenvalues: 4.50000e-02 5.00000e-02 5.00000e-02 ... 3.26000e-01 3.26000e-01 9.99547e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.011945  -0.000000  -0.018808   -0.011945 -0.000000 -0.018808
   C   1.190323  -0.000000   0.012081   -0.019677 -0.000000  0.012081
   H   1.794405  -0.000000   0.936491    0.029405 -0.000000 -0.024798
   H   1.766847  -0.000000  -0.929763    0.001847 -0.000000  0.031525
converged SCF energy = -114.549379095696
--------------- SymAdaptedRKS_Scanner gradients ---------------
         x                y                z
0 O    -0.0052937044     0.0000000000    -0.0000000000
1 C     0.0054168995     0.0000000000     0.0000000000
2 H    -0.0000579020    -0.0000000000    -0.0015945915
3 H    -0.0000579020     0.0000000000     0.0015945915
----------------------------------------------
cycle 2: E = -114.549379096  dE = -0.000626921  norm(grad) = 0.00790305


Step    1 : Displace = 2.961e-02/3.854e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 3.952e-03/5.417e-03 (rms/max) E (change) = -114.5493790957 (-6.269e-04) Quality = 0.953
Hessian Eigenvalues: 4.50000e-02 4.99890e-02 5.00000e-02 ... 3.26000e-01 3.44245e-01 9.09278e-01



Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.008829  -0.000000  -0.020254    0.003116 -0.000000 -0.020254
   C   1.190535  -0.000000   0.011377    0.000211 -0.000000  0.011377
   H   1.789828  -0.000000   0.941151    0.009202 -0.000000  0.008024
   H   1.767790  -0.000000  -0.932275   -0.012836 -0.000000  0.000852
converged SCF energy = -114.549406851134
--------------- SymAdaptedRKS_Scanner gradients ---------------
         x                y                z
0 O    -0.0004149866     0.0000000000    -0.0000000000
1 C     0.0007950761     0.0000000000     0.0000000000
2 H    -0.0001863609    -0.0000000000    -0.0003109902
3 H    -0.0001863609     0.0000000000     0.0003109902
----------------------------------------------
cycle 3: E = -114.549406851  dE = -2.77554e-05  norm(grad) = 0.00103308


Step    2 : Displace = 3.330e-03/4.181e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 5.165e-04/7.951e-04 (rms/max) E (change) = -114.5494068511 (-2.776e-05) Quality = 1.104
Hessian Eigenvalues: 4.50000e-02 4.98595e-02 5.00000e-02 ... 3.10291e-01 3.26001e-01 8.80885e-01



Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.008906  -0.000000  -0.020688   -0.000077 -0.000000 -0.020688
   C   1.190142  -0.000000   0.011435   -0.000393 -0.000000  0.011435
   H   1.789677  -0.000000   0.941826    0.010868 -0.000000  0.005113
   H   1.768060  -0.000000  -0.932573   -0.010750 -0.000000  0.004140
converged SCF energy = -114.549407439867
--------------- SymAdaptedRKS_Scanner gradients ---------------
         x                y                z
0 O     0.0000031044     0.0000000000     0.0000000000
1 C     0.0000209565     0.0000000000     0.0000000000
2 H    -0.0000083603    -0.0000000000    -0.0000073277
3 H    -0.0000083603     0.0000000000     0.0000073277
----------------------------------------------
cycle 4: E = -114.54940744  dE = -5.88733e-07  norm(grad) = 2.63816e-05


Step    3 : Displace = 3.941e-04/5.405e-04 (rms/max) Trust = 2.000e-01 (+) Grad = 1.319e-05/2.096e-05 (rms/max) E (change) = -114.5494074399 (-5.887e-07) Quality = 1.030
Hessian Eigenvalues: 4.50000e-02 4.98595e-02 5.00000e-02 ... 3.10291e-01 3.26001e-01 8.80885e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 30.329 seconds


In [ ]:
hcoh_ccsd_opt = ccsd_hcoh.Gradients().optimizer(solver='geomeTRIC').kernel()

geometric-optimize called with the following command line:
/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-5a1718ce-10f0-4cf6-bc84-581707384c7f.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   C   1.210000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   1.765000   0.000000   0.961288    0.000000  0.000000  0.000000
   H   1.765000  -0.000000  -0.961288    0.000000  0.000000  0.000000
converged SCF energy = -113.910581741567
E(CCSD_Scanner) = -114.344863310445  E_corr = -0.4342815688780536
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0141877436     0.0000000000    -0.0000000000
1 C     0.0149345135     0.0000000000     0.0000000000
2 H    -0.0003733849     0.0000000000     0.0112128224
3 H    -0.0003733849    -0.0000000000    -0.0112128224
----------------------------------------------
cycle 1: E = -114.34486331  dE = -114.345  norm(grad) = 0.0260013


Step    0 : Gradient = 1.300e-02/1.493e-02 (rms/max) Energy = -114.3448633104
Hessian Eigenvalues: 4.50000e-02 5.00000e-02 5.00000e-02 ... 3.26000e-01 3.26000e-01 9.99547e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.008410  -0.000000  -0.018023   -0.008410 -0.000000 -0.018023
   C   1.193715  -0.000000   0.011559   -0.016285 -0.000000  0.011559
   H   1.790557   0.000000   0.929055    0.025557 -0.000000 -0.032233
   H   1.764138  -0.000000  -0.922591   -0.000862  0.000000  0.038697
converged SCF energy = -113.912132830521
E(CCSD_Scanner) = -114.345639901345  E_corr = -0.4335070708237566
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0038764960     0.0000000000    -0.0000000000
1 C     0.0032433323     0.0000000000     0.0000000000
2 H     0.0003165818    -0.0000000000    -0.0017498041
3 H     0.0003165818     0.0000000000     0.0017498041
----------------------------------------------
cycle 2: E = -114.345639901  dE = -0.000776591  norm(grad) = 0.0056454


Step    1 : Displace = 3.156e-02/4.114e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 2.823e-03/3.876e-03 (rms/max) E (change) = -114.3456399013 (-7.766e-04) Quality = 0.901
Hessian Eigenvalues: 4.50000e-02 4.99966e-02 5.00000e-02 ... 3.25998e-01 3.43126e-01 9.39973e-01



Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.004599  -0.000000  -0.017274    0.003811 -0.000000 -0.017274
   C   1.195624  -0.000000   0.009913    0.001909 -0.000000  0.009913
   H   1.784463   0.000000   0.934472    0.007116 -0.000000  0.008649
   H   1.764512  -0.000000  -0.927110   -0.012836  0.000000 -0.001287
converged SCF energy = -113.912303659576
E(CCSD_Scanner) = -114.3456657984998  E_corr = -0.4333621389236063
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0001914506     0.0000000000    -0.0000000000
1 C     0.0006207022     0.0000000000     0.0000000000
2 H    -0.0002146258    -0.0000000000    -0.0001299379
3 H    -0.0002146258     0.0000000000     0.0001299379
----------------------------------------------
cycle 3: E = -114.345665798  dE = -2.58972e-05  norm(grad) = 0.000740149


Step    2 : Displace = 4.993e-03/6.501e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 3.701e-04/6.207e-04 (rms/max) E (change) = -114.3456657985 (-2.590e-05) Quality = 0.985
Hessian Eigenvalues: 4.50000e-02 4.99278e-02 5.00000e-02 ... 3.22271e-01 3.26007e-01 9.07666e-01



Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.004879  -0.000000  -0.017737   -0.000281 -0.000000 -0.017737
   C   1.195195  -0.000000   0.010109   -0.000429 -0.000000  0.010109
   H   1.784909   0.000000   0.934571    0.010422  0.000000  0.003780
   H   1.764775  -0.000000  -0.926943   -0.009712  0.000000  0.003848
converged SCF energy = -113.912307973273
E(CCSD_Scanner) = -114.3456662452603  E_corr = -0.4333582719870867
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0000646707     0.0000000000    -0.0000000000
1 C     0.0000952934     0.0000000000     0.0000000000
2 H    -0.0000153114    -0.0000000000    -0.0000029858
3 H    -0.0000153114     0.0000000000     0.0000029858
----------------------------------------------
cycle 4: E = -114.345666245  dE = -4.4676e-07  norm(grad) = 0.00011726


Step    3 : Displace = 4.322e-04/5.766e-04 (rms/max) Trust = 2.000e-01 (+) Grad = 5.863e-05/9.529e-05 (rms/max) E (change) = -114.3456662453 (-4.468e-07) Quality = 1.328
Hessian Eigenvalues: 4.50000e-02 4.99278e-02 5.00000e-02 ... 3.22271e-01 3.26007e-01 9.07666e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 130.794 seconds


In [ ]:
print(hcoh_ccsd_opt.tostring())

O          -0.00488        0.00000       -0.00000
C           1.19519        0.00000        0.00000
H           1.77484        0.00000        0.93076
H           1.77484       -0.00000       -0.93076


In [ ]:
hcoh_opt = gto.Mole(atom=hcoh_hf_opt_geomtric.tostring(), unit='Angstrom', basis='ccpvtz', charge=0, spin=0)
hcoh_opt.build()
hf_freq = scf.RHF(hcoh_opt).run()
hcoh_eph = eph.EPH(hf_freq)
start_hf_eph = time.time()
ephmat, omega = hcoh_eph.kernel()
print(f'HF electron-phonon coupling done in {(time.time()-start_hf_eph):.3f} seconds')
omega

converged SCF energy = -113.913225324369
HF electron-phonon coupling done in 52.138 seconds


array([0.0143573 , 0.01405037, 0.00907648, 0.00752758, 0.00623918,
       0.00608878])

In [ ]:
omega*219474.63

array([3151.06293881, 3083.69943781, 1992.05680977, 1652.11343237,
       1369.34213752, 1336.33361884])

In [ ]:
hcoh_opt = gto.Mole(atom=hcoh_b3lyp_opt.tostring(), unit='Angstrom', basis='ccpvtz', charge=0, spin=0)
hcoh_opt.build()
ks_freq = dft.RKS(hcoh_opt, xc='B3LYP').run()
hcoh_eph = eph.EPH(ks_freq)
start_ks_eph = time.time()
ephmat, omega = hcoh_eph.kernel()
print(f'HF electron-phonon coupling done in {(time.time()-start_ks_eph):.3f} seconds')
omega

converged SCF energy = -114.54940743995
HF electron-phonon coupling done in 73.438 seconds


array([0.01335299, 0.013111  , 0.00828241, 0.00699951, 0.0057738 ,
       0.00547556])

In [ ]:
omega*219474.63

array([2930.64263018, 2877.53158117, 1817.7788907 , 1536.21383822,
       1267.20275743, 1201.74702272])

In [ ]:
hcoh_opt = gto.Mole(atom=hcoh_b3lyp_opt.tostring(), unit='Angstrom', basis='ccpvtz', charge=0, spin=0)
hcoh_opt.build()
hcoh_ks = dft.RKS(hcoh_opt, xc='B3LYP').run()
hcoh_mp2_opt = mp.MP2(hcoh_ks).run()
hcoh_ccsd_opt = cc.CCSD(hcoh_ks.to_hf()).run()
hcoh_ccsd_opt.ccsd_t()

converged SCF energy = -114.54940743995
E(MP2) = -115.151385823635  E_corr = -0.601978383684932
E(SCS-MP2) = -115.146207717614  E_corr = -0.596800277663408
E(CCSD) = -114.3439940977523  E_corr = -0.4482897620088517
CCSD(T) correction = -0.0192242098218655


np.float64(-0.019224209821865523)

Д/з до 03.04.2025


1.   Рассчитать методами (ограниченными и неограниченными) MP2 и CCSD/CCSD(T) молекулу из 1 задания Д/з №2 с теми же 3 базисными наборами. Сравнить результаты расчетов и время выполнения.
2.   Оптимизировать молекулу из 1 задания Д/з №2 любыми 3 функционалами плотности с теми же 3 базисными наборами. Сравнить результаты расчетов и время выполнения.